# Baseline Model

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor

In [2]:
dataset=f"data/cleaned_tabular.csv"
df=pd.read_csv(dataset)

In [3]:
df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,12.501142,0.677402,0.178963,-0.290276,-0.144952,0.922943,0,-0.306964,-0.626000,-0.557611,-0.345663,-0.207964,12.581985,-0.900034,0.192759,-0.473911,-0.129791
1,12.409018,-0.394132,0.505667,-0.521813,-0.311135,0.922943,0,-0.306964,0.908842,-0.557611,0.709771,-0.207964,12.591252,-1.137139,0.192759,-0.385919,-0.339019
2,12.206078,0.677402,0.505667,-0.389506,-0.160457,0.922943,0,-0.306964,-0.626000,0.296350,0.777864,-0.207964,12.531108,-2.098571,-0.706669,-0.165941,-0.196068
3,12.772806,-1.465666,0.178963,-0.918734,-0.364787,0.922943,0,-0.306964,-0.626000,-0.557611,1.288558,-0.207964,13.257362,-0.206791,1.006527,-1.089851,-0.445025
4,12.354497,-0.394132,-0.147741,-0.874632,-0.038936,-0.918626,0,-0.306964,-0.626000,-0.557611,0.777864,-0.207964,12.605452,-1.367738,0.999388,-0.576568,-0.173196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16204,12.842652,-0.394132,-0.801150,-1.183348,-0.204862,-0.918626,0,-0.306964,-0.626000,-0.557611,-0.822311,-0.207964,12.987940,1.111021,-0.749499,-1.441817,-0.214930
16205,12.899097,-0.394132,0.505667,1.117691,-0.254109,0.922943,0,-0.306964,-0.626000,0.296350,1.458789,-0.207964,12.523093,-1.903393,-0.963648,1.384176,-0.280428
16206,13.262127,-0.394132,0.505667,0.051517,-0.259827,0.922943,0,-0.306964,-0.626000,-0.557611,1.118326,-0.207964,13.330295,0.869579,1.299198,-0.429915,-0.374478
16207,12.409018,-2.537201,-1.781262,-1.866934,0.003408,-0.918626,0,-0.306964,-0.626000,-2.265534,-0.277571,-0.207964,12.356950,-0.576183,-0.778052,-1.192508,0.084078


In [4]:
X=df.drop(columns=["price"])
y=df["price"]
X.shape,y.shape

((16209, 16), (16209,))

In [5]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=42)

In [6]:
ridge=Ridge(alpha=1.0)
ridge.fit(X_train,y_train)

Ridge()

In [7]:
pred=ridge.predict(X_val)
rsme=np.sqrt(mean_squared_error(np.expm1(y_val),np.expm1(pred)))
r2=r2_score(np.expm1(y_val),np.expm1(pred))
print(rsme)
print(r2)

140547.19015190302
0.8425872068208815


In [10]:
xgb=XGBRegressor(
    n_estimators=500,max_depth=6,learning_rate=0.05,subsample=0.8,colsample_bytree=0.8,random_state=42,n_jobs=-1
)
xgb.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=-1, num_parallel_tree=None, ...)

In [11]:
pred=xgb.predict(X_val)
rsme=np.sqrt(mean_squared_error(np.expm1(y_val),np.expm1(pred)))
r2=r2_score(np.expm1(y_val),np.expm1(pred))
print(rsme,"by xgb")
print(r2,"by xgb")

113987.65198668418 by xgb
0.8964592501717463 by xgb


we can see a maximum R2 score with XGB Regressor

# Multimodal

In [12]:
from operator import index
importances=pd.Series(xgb.feature_importances_,index=X.columns).sort_values()
importances

bedrooms         0.005198
floors           0.005604
sqft_lot15       0.009026
yr_renovated     0.009146
sqft_lot         0.009321
yr_built         0.010169
condition        0.012287
bathrooms        0.013413
sqft_living15    0.018584
long             0.019097
lat              0.045946
view             0.054163
waterfront       0.062279
sqft_living      0.109075
grade            0.281193
zipcode          0.335500
dtype: float32

Now lets us build a multi-model by useing tabular data and images

In [13]:
# image embeddings extraction
import os
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
from torchvision import models,transforms

c:\Users\Abhinav Shivanadri\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
image_transform=transforms.Compose([
    transforms.Resize((224,224)),transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.485,0.406],std=[0.229,0.224,0.225])
])

In [15]:
DEVICE="cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cpu'

In [16]:
resnet=models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
resnet.fc=nn.Identity()
for param in resnet.parameters():
  param.requires_grad=False
resnet=resnet.to(DEVICE)
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
IMAGE_DIR = "data/arcgis_images"
image_files=sorted(os.listdir(IMAGE_DIR))
len(image_files)

16209

In [18]:
import torch
from torch.utils.data import Dataset,DataLoader
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self,image_dir,transform):
        self.image_dir=image_dir
        self.files=sorted(os.listdir(image_dir))
        self.transform=transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self,idx):
        img_path=os.path.join(self.image_dir,self.files[idx])
        img=Image.open(img_path).convert("RGB")
        return self.transform(img),self.files[idx]


In [20]:
from tqdm import tqdm
IMAGE_DIR="data/arcgis_images"
dataset=ImageDataset(IMAGE_DIR, image_transform)
loader=DataLoader(dataset, batch_size=32, shuffle=False)
EMBED_DIR=f"data/image_embeddings"
with torch.no_grad():
    for imgs,names in tqdm(loader):
        imgs=imgs.to(DEVICE)
        feats=resnet(imgs).squeeze(-1).squeeze(-1)
        feats=feats.cpu().numpy()

        for f,n in zip(feats, names):
            np.save(
                os.path.join(EMBED_DIR, n.replace(".jpg", ".npy")),
                f
            )


100%|██████████| 507/507 [12:06<00:00,  1.43s/it]


In [21]:
BASE_PATH = "data"
X_tab = pd.read_csv(f"{BASE_PATH}/cleaned_tabular.csv")
embeddings = []
EMB_DIR = f"{BASE_PATH}/image_embeddings"
for i in range(len(X_tab)):
    emb_path = os.path.join(EMB_DIR, f"{i}.npy")

    if not os.path.exists(emb_path):
        raise FileNotFoundError(f"Missing embedding: {emb_path}")

    emb = np.load(emb_path)
    embeddings.append(emb)

X_img = np.vstack(embeddings)
print(X_img.shape)


(16209, 512)


In [22]:
print("Tabular rows:", X_tab.shape[0])
print("Image rows:", X_img.shape[0])


Tabular rows: 16209
Image rows: 16209


In [23]:
from sklearn.decomposition import PCA

pca = PCA(
    n_components=64,
    random_state=42,
    svd_solver="randomized"
)

X_img_pca = pca.fit_transform(X_img)

X_img_pca.shape


(16209, 64)

In [24]:
y=X_tab["price"]
X_tab=X_tab.drop(columns=["price"])

In [25]:
X_tab.shape,y.shape

((16209, 16), (16209,))

In [26]:
assert X_tab.shape[0] == X_img_pca.shape[0], "Row mismatch!"


In [27]:
import numpy as np

X_fused = np.hstack([
    X_tab.values,
    X_img_pca
])

X_fused.shape


(16209, 80)

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_fused,
    y,
    test_size=0.2,
    random_state=42
)


In [29]:
from xgboost import XGBRegressor

mm_model = XGBRegressor(
    n_estimators=700,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

mm_model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=700,
             n_jobs=-1, num_parallel_tree=None, ...)

In [30]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

preds = mm_model.predict(X_val)

rmse = np.sqrt(mean_squared_error(
    np.expm1(y_val),
    np.expm1(preds)
))

r2 = r2_score(
    np.expm1(y_val),
    np.expm1(preds)
)

print("===== MULTIMODAL RESULTS =====")
print(f"RMSE: {rmse:,.0f}")
print(f"R²  : {r2:.4f}")


===== MULTIMODAL RESULTS =====
RMSE: 115,628
R²  : 0.8935
